# NeuroSym-KG: Quick Start Guide

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/your-org/neurosym-kg/blob/main/notebooks/01_quickstart.ipynb)

This notebook introduces the NeuroSym-KG framework for neuro-symbolic reasoning over knowledge graphs.

**What you'll learn:**
- How to install NeuroSym-KG
- Creating your first knowledge graph
- Running a reasoning query
- Understanding the output

## 1. Installation

First, let's install NeuroSym-KG and its dependencies. This cell only needs to run once per Colab session.

In [ ]:
# Install NeuroSym-KG from GitHub
!pip install -q git+https://github.com/your-org/neurosym-kg.git

# Or if you have the package locally/uploaded:
# !pip install -q neurosym-kg

# For this demo, let's install the core dependencies directly:
!pip install -q pydantic pydantic-settings httpx tenacity networkx numpy tqdm

In [ ]:
# If running from source (e.g., uploaded to Colab), add to path:
import sys
sys.path.insert(0, '/content/neurosym-kg')  # Adjust path as needed

# Verify installation
try:
    import neurosym_kg
    print(f"✅ NeuroSym-KG v{neurosym_kg.__version__} installed successfully!")
except ImportError as e:
    print(f"❌ Installation issue: {e}")
    print("Please run the installation cell above first.")

## 2. Set Up API Keys (Optional)

If you want to use OpenAI or other LLM providers, set up your API keys.

**Recommended**: Use Colab's Secrets feature (🔑 icon in left sidebar) to store keys securely.

In [ ]:
import os

# Option 1: Use Colab Secrets (recommended)
try:
    from google.colab import userdata
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
    print("✅ API key loaded from Colab Secrets")
except:
    pass

# Option 2: Set directly (not recommended for shared notebooks)
# os.environ['OPENAI_API_KEY'] = 'sk-your-key-here'

# For this quickstart, we'll use a Mock LLM that doesn't need an API key
print("ℹ️ This tutorial uses MockLLM - no API key required!")

## 3. Create Your First Knowledge Graph

Let's create a simple knowledge graph about scientists.

In [ ]:
from neurosym_kg import InMemoryKG, Triple

# Create an in-memory knowledge graph
kg = InMemoryKG(name="Scientists KG")

# Add facts as triples (subject, predicate, object)
kg.add_triples([
    # Albert Einstein
    Triple("Albert_Einstein", "born_in", "Ulm"),
    Triple("Albert_Einstein", "field", "Physics"),
    Triple("Albert_Einstein", "nationality", "German"),
    Triple("Albert_Einstein", "award", "Nobel_Prize_Physics_1921"),
    Triple("Albert_Einstein", "known_for", "Theory_of_Relativity"),
    
    # Locations
    Triple("Ulm", "located_in", "Germany"),
    Triple("Ulm", "type", "City"),
    
    # Marie Curie
    Triple("Marie_Curie", "born_in", "Warsaw"),
    Triple("Marie_Curie", "field", "Physics"),
    Triple("Marie_Curie", "field", "Chemistry"),
    Triple("Marie_Curie", "nationality", "Polish"),
    Triple("Marie_Curie", "award", "Nobel_Prize_Physics_1903"),
    Triple("Marie_Curie", "award", "Nobel_Prize_Chemistry_1911"),
    
    # More locations
    Triple("Warsaw", "located_in", "Poland"),
    Triple("Germany", "type", "Country"),
    Triple("Poland", "type", "Country"),
])

# Print summary
print(kg.summary())

## 4. Explore the Knowledge Graph

Let's query our knowledge graph directly before using a reasoner.

In [ ]:
# Get information about Einstein
print("=" * 50)
print("Facts about Einstein:")
print("=" * 50)

einstein_facts = kg.get_neighbors("Albert_Einstein", direction="outgoing")
for fact in einstein_facts:
    print(f"  {fact.to_text()}")

print("\n" + "=" * 50)
print("Who works in Physics?")
print("=" * 50)

physics_triples = kg.get_triples(predicate="field", obj="Physics")
for t in physics_triples:
    print(f"  {t.subject_id}")

In [ ]:
# Find paths between entities
print("Path from Einstein to Germany:")
paths = kg.find_paths("Albert_Einstein", "Germany", max_hops=2)

for i, path in enumerate(paths):
    print(f"\nPath {i+1}:")
    for triple in path:
        print(f"  → {triple.to_text()}")

## 5. Set Up the Reasoner

Now let's create a reasoner that combines the KG with an LLM to answer questions.

In [ ]:
from neurosym_kg import MockLLMBackend, ThinkOnGraphReasoner

# Create a mock LLM for demonstration
# In production, you'd use OpenAIBackend or AnthropicBackend
llm = MockLLMBackend(default_response="Unknown")

# Configure the mock LLM with some responses
llm.add_response(r".*Extract.*entities.*Einstein.*", "Albert_Einstein")
llm.add_response(r".*Extract.*entities.*Curie.*", "Marie_Curie")
llm.add_response(r".*Extract.*entities.*born.*", "Albert_Einstein")
llm.add_response(r".*relevant relations.*", "born_in\nfield")
llm.add_response(r".*enough information.*", "YES")
llm.add_response(r".*answer.*born.*", "Albert Einstein was born in Ulm, Germany.")
llm.add_response(r".*answer.*field.*", "Albert Einstein worked in Physics.")

# Create the Think-on-Graph reasoner
reasoner = ThinkOnGraphReasoner(
    kg=kg,
    llm=llm,
    max_depth=2,      # Maximum hops in the graph
    beam_width=3,     # Number of paths to explore
    verbose=True      # Print debug info
)

print("✅ Reasoner initialized!")

## 6. Ask Questions!

Now let's use the reasoner to answer questions about our knowledge graph.

In [ ]:
# Ask a question
question = "Where was Einstein born?"
print(f"❓ Question: {question}")
print("=" * 50)

result = reasoner.reason(question)

print("\n" + "=" * 50)
print("📝 Results:")
print("=" * 50)
print(f"Answer: {result.answer}")
print(f"Confidence: {result.confidence:.2%}")
print(f"Status: {result.status.value}")
print(f"Latency: {result.latency_ms:.0f}ms")

if result.paths:
    print(f"\nReasoning Paths ({len(result.paths)} found):")
    for i, path in enumerate(result.paths[:3]):
        print(f"  Path {i+1}: {path.to_text()}")

In [ ]:
# Try another question
question2 = "What field did Einstein work in?"
print(f"❓ Question: {question2}")
print("=" * 50)

result2 = reasoner.reason(question2)

print(f"\n📝 Answer: {result2.answer}")
print(f"   Confidence: {result2.confidence:.2%}")

## 7. Check Reasoner Statistics

The reasoner tracks usage statistics.

In [ ]:
stats = reasoner.stats

print("📊 Reasoner Statistics")
print("=" * 30)
print(f"Total queries: {stats['queries']}")
print(f"KG calls: {stats['kg_calls']}")
print(f"LLM calls: {stats['llm_calls']}")
print(f"Successful: {stats['successful']}")
print(f"Failed: {stats['failed']}")

if stats['queries'] > 0:
    print(f"Avg latency: {stats['total_latency_ms']/stats['queries']:.0f}ms")

## 8. Using with Real LLM (OpenAI)

To use with a real LLM, just swap the backend:

In [ ]:
# Uncomment and run if you have an OpenAI API key set up
"""
from neurosym_kg import OpenAIBackend

# Create real LLM backend
real_llm = OpenAIBackend(
    model="gpt-4o-mini",  # or "gpt-4o" for better results
    temperature=0.0       # Deterministic outputs
)

# Create reasoner with real LLM
real_reasoner = ThinkOnGraphReasoner(
    kg=kg,
    llm=real_llm,
    max_depth=3,
    beam_width=5,
)

# Ask a question
result = real_reasoner.reason("Where was Einstein born?")
print(f"Answer: {result.answer}")
"""
print("💡 Uncomment the code above to use with OpenAI")

## 🎉 Next Steps

Congratulations! You've learned the basics of NeuroSym-KG. Here's what to explore next:

1. **[02_comparing_reasoners.ipynb](./02_comparing_reasoners.ipynb)** - Compare different reasoning paradigms
2. **[03_custom_kg.ipynb](./03_custom_kg.ipynb)** - Build domain-specific knowledge graphs
3. **[04_evaluation.ipynb](./04_evaluation.ipynb)** - Run benchmarks and evaluate performance
4. **[05_symbolic_reasoning.ipynb](./05_symbolic_reasoning.ipynb)** - Add rule-based constraints

### Resources

- [GitHub Repository](https://github.com/your-org/neurosym-kg)
- [API Documentation](https://neurosym-kg.readthedocs.io)
- [Research Papers](#) (ToG, RoG, GraphRAG)